In [1]:
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
import sklearn
from sklearn import metrics
from sklearn.metrics import precision_score, recall_score, f1_score
from pandas import Series, DataFrame
import scipy
from scipy import stats
import math
import csv
from sklearn import feature_extraction

In [2]:
data_train = pd.read_csv("Dataset-football-train.txt", delimiter="\t")

In [3]:
data_test = pd.read_csv("Dataset-football-test.txt", delimiter="\t")

In [4]:
i = 0
for item in data_train["Date"]:
    if item[0] == "9":
        data_train["Date"][i] = "sept"
    elif item[1] == "0":
        data_train["Date"][i] = "oct"
    else:
        data_train["Date"][i] = "nov"
    i += 1

/Users/kailee/anaconda/envs/py36/lib/python3.6/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.
/Users/kailee/anaconda/envs/py36/lib/python3.6/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/Users/kailee/anaconda/envs/py36/lib/python3.6/site-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [5]:
i = 0
for item in data_test["Date"]:
    if item[0] == "9":
        data_test["Date"][i] = "sept"
    elif item[1] == "0":
        data_test["Date"][i] = "oct"
    else:
        data_test["Date"][i] = "nov"
    i += 1

/Users/kailee/anaconda/envs/py36/lib/python3.6/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.
/Users/kailee/anaconda/envs/py36/lib/python3.6/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/Users/kailee/anaconda/envs/py36/lib/python3.6/site-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [6]:
def prior_prob(column, label):
    label_count = 0
    for item in column:
        if item == label:
            label_count += 1
    return (float(label_count)/len(column))

In [7]:
def likelihood(column, label, predict_column, pos_label):
    count = 0
    for index, value in enumerate(column):
        if predict_column[index] == pos_label:
            if value == label:
                count += 1
    pos_count = 0
    for item in predict_column:
        if item == pos_label:
            pos_count += 1
    return float(count)/pos_count

In [8]:
likelihood(list(data_train["Media"]), "NBC", list(data_train["Label"]), "Win")

0.7142857142857143

In [9]:
def evidence(column, label):
    label_count = 0
    for item in column:
        if item == label:
            label_count += 1
    return (float(label_count)/len(column))

In [10]:
def post_prob(prior, likelihood,evidence):
    return (prior*float(likelihood))/evidence

In [11]:
def train(dataframe, predict_column, pos_label, column_names):
    probabilities = {}
    pos_prior = prior_prob(predict_column, pos_label)
    for name in column_names:
        column = list(dataframe[name])
        values = set()
        for value in column:
            values.add(value)
        for unique_value in values:
            likeli = likelihood(column, unique_value, predict_column, pos_label)
            evi = evidence(column, unique_value)
            po_prob = post_prob(pos_prior, likeli, evi)
            probabilities[unique_value] = po_prob
    print(probabilities)
    return probabilities
        
    

In [12]:
data_train.head()

,ID,Date,Opponent,Is_Home_or_Away,Is_Opponent_in_AP25_Preseason,Media,Label
0,1,sept,Texas,Home,Out,NBC,Win
1,2,sept,Virginia,Away,Out,ABC,Win
2,3,sept,Georgia Tech,Home,In,NBC,Win
3,4,sept,UMass,Home,Out,NBC,Win
4,5,oct,Clemson,Away,In,ABC,Lose


In [13]:
predicting_column = list(data_train["Label"])

In [14]:
data_train.drop(["Label", "ID"], axis=1, inplace=True)

In [15]:
test_predict_column = list(data_test["Label"])

In [16]:
data_test.drop(["Label", "ID"], axis=1, inplace=True)

In [17]:
list_names = ["Date", "Opponent", "Is_Home_or_Away", "Is_Opponent_in_AP25_Preseason", "Media"]

In [18]:
train_output = train(data_train, predicting_column,"Win", list_names)

{'nov': 0.5, 'oct': 0.6250000000000001, 'sept': 0.6250000000000001, 'Syracuse': 1.0, 'Virginia': 1.0, 'Duke': 0.0, 'Texas': 0.5, 'Navy': 0.5, 'Clemson': 0.0, 'Boston College': 1.0, 'Army': 1.0, 'Miami Florida': 1.0, 'PITT': 1.0, 'Temple': 1.0, 'Wake Forest': 1.0, 'North Carolina State': 0.0, 'Michigan State': 0.0, 'Virginia Tech': 0.0, 'Nevada': 1.0, 'Stanford': 0.0, 'Georgia Tech': 1.0, 'UMass': 1.0, 'USC': 0.5, 'Home': 0.6666666666666667, 'Away': 0.4444444444444444, 'In': 0.2857142857142857, 'Out': 0.7058823529411764, 'NBC': 0.7142857142857143, 'CBS': 0.0, 'FOX': 0.0, 'ABC': 0.42857142857142855, 'ESPN': 1.0}


In [19]:
def test(dataframe, column_names, train_output):
    predicted = []
    for index, row in dataframe.iterrows():
        row_prob = 1
        for name in column_names:
            value = row[name]
            try:
                prob = train_output[value]
            except:
                prob = 0
            row_prob = row_prob * prob
        if row_prob >= .5:
            predicted.append("Win")
        else:
            predicted.append("Lose")
    print(predicted)
    return predicted
            

In [20]:
predicted = test(data_test, list_names, train_output)

['Lose', 'Lose', 'Lose', 'Lose', 'Lose', 'Lose', 'Lose', 'Lose', 'Lose', 'Lose', 'Lose', 'Lose']


In [21]:
precision_score(test_predict_column,predicted, pos_label="Lose")

0.25

In [22]:
recall_score(test_predict_column,predicted, pos_label="Lose")

1.0

In [23]:
f1_score(test_predict_column,predicted, pos_label="Lose")

0.4